In [1]:
import os
import sys
import glob
import time
import random
import pandas as pd
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from lib.config import *
from lib.encoder.ffn import FFNEncoder, FFNDecoder
from lib.encoder.vqvae import VQVAE, VectorQuantizer
from lib.train.run_autoencoder_training import AutoencoderTrainer
from lib.utils.pose import get_pose_estimation
import datetime

In [2]:
import os 

os.cpu_count()

8

In [3]:
mp4_files = glob.glob("dataset/corpus/*.mp4")
mp4_files

['dataset/corpus/GEREK _1.mp4',
 'dataset/corpus/I╠çLGINC╠ğ_0.mp4',
 'dataset/corpus/MEMUR_1.mp4',
 'dataset/corpus/BAS╠ğPARMAK_0.mp4',
 'dataset/corpus/YILDIZ_0.mp4',
 'dataset/corpus/ZARF_0.mp4',
 'dataset/corpus/SERVIS_1.mp4',
 'dataset/corpus/HAKLI_0.mp4',
 'dataset/corpus/KIM_1.mp4',
 'dataset/corpus/I╠çKISI_0.mp4',
 'dataset/corpus/DIG╠åER_0.mp4',
 'dataset/corpus/BAYRAK_0.mp4',
 'dataset/corpus/I╠çNS╠ğALLAH_1.mp4',
 'dataset/corpus/HOPARLO╠êR_0.mp4',
 'dataset/corpus/I╠çMZALAMAK_0.mp4',
 'dataset/corpus/GARANTI_0.mp4',
 'dataset/corpus/KUSURA BAKMAMAK_1.mp4',
 'dataset/corpus/SAC╠ğ_0.mp4',
 'dataset/corpus/VERGI _1.mp4',
 'dataset/corpus/YAVAS╠ğLAMAK_0.mp4',
 'dataset/corpus/BAG╠åIMSIZ _0.mp4',
 'dataset/corpus/GO╠êNU╠êL_0.mp4',
 'dataset/corpus/TREN_1.mp4',
 'dataset/corpus/C╠ğAY_1.mp4',
 'dataset/corpus/UC╠ğAK_0.mp4',
 'dataset/corpus/ALET_0.mp4',
 'dataset/corpus/PRATIK_0.mp4',
 'dataset/corpus/I╠çC╠ğIN_0.mp4',
 'dataset/corpus/BIRA_1.mp4',
 'dataset/corpus/

In [4]:
GLOBAL_CONFIG

namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False,
          MEDIAPIPE_MODEL_COMPLEXITY=1,
          MEDIAPIPE_SMOOTH_LANDMARKS=True,
          MEDIAPIPE_ENABLE_SEGMENTATION=False,
          MEDIAPIPE_SMOOTH_SEGMENTATION=True,
          MEDIAPIPE_REFINE_FACE_LANDMARKS=False,
          MEDIAPIPE_MIN_DETECTION_CONFIDENCE=0.5,
          MEDIAPIPE_MIN_TRACKING_CONFIDENCE=0.5,
          MODEL_ENCODER_INPUT_DIM=225,
          MODEL_ENCODER_HIDDEN_DIM=256,
          MODEL_ENCODER_OUTPUT_DIM=768,
          MODEL_VQ_NUM_EMBS=10000,
          MODEL_VQ_EMBED_DIM=768,
          MODEL_VQ_COMMITMENT_COST=0.25,
          MODEL_DECODER_INPUT_DIM=768,
          MODEL_DECODER_HIDDEN_DIM=256,
          MODEL_DECODER_OUTPUT_DIM=225,
          NUM_EPOCHS=100,
          BATCH_SIZE=32,
          LEARNING_RATE=1e-05)

In [5]:
GLOBAL_CONFIG.MEDIAPIPE_MODEL_COMPLEXITY = 2
GLOBAL_CONFIG

namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False,
          MEDIAPIPE_MODEL_COMPLEXITY=2,
          MEDIAPIPE_SMOOTH_LANDMARKS=True,
          MEDIAPIPE_ENABLE_SEGMENTATION=False,
          MEDIAPIPE_SMOOTH_SEGMENTATION=True,
          MEDIAPIPE_REFINE_FACE_LANDMARKS=False,
          MEDIAPIPE_MIN_DETECTION_CONFIDENCE=0.5,
          MEDIAPIPE_MIN_TRACKING_CONFIDENCE=0.5,
          MODEL_ENCODER_INPUT_DIM=225,
          MODEL_ENCODER_HIDDEN_DIM=256,
          MODEL_ENCODER_OUTPUT_DIM=768,
          MODEL_VQ_NUM_EMBS=10000,
          MODEL_VQ_EMBED_DIM=768,
          MODEL_VQ_COMMITMENT_COST=0.25,
          MODEL_DECODER_INPUT_DIM=768,
          MODEL_DECODER_HIDDEN_DIM=256,
          MODEL_DECODER_OUTPUT_DIM=225,
          NUM_EPOCHS=100,
          BATCH_SIZE=32,
          LEARNING_RATE=1e-05)

In [6]:
def get_analyze_statistics(video_name: str) -> list:
    """Returns analyze statistics as list.

    Args:
        video_name (str):

    Returns:
        list: statistics list
            : video_name
            : num_of_frames_total
            : num_of_frames_missing_pose
            : num_of_frames_missing_left_hand
            : num_of_frames_missing_right_hand
            
    """
    
    # print(f"Analyzing: {video_name}")

    estimation, info = get_pose_estimation(video_name, with_info=True)

    pd_info = pd.DataFrame(info, columns=["video_name", "info", "value"])

    num_of_frames_total = pd_info[pd_info['info']
                                  == 'total_number_of_frames']["value"].iloc[0]
    num_of_frames_missing_pose = 0
    num_of_frames_missing_left_hand = 0
    num_of_frames_missing_right_hand = 0

    if "missing_pose" in pd_info["info"].to_list():
        num_of_frames_missing_pose = pd_info.value_counts("info")[
            "missing_pose"]

    if "missing_left_hand" in pd_info["info"].to_list():
        num_of_frames_missing_left_hand = pd_info.value_counts("info")[
            "missing_left_hand"]

    if "missing_right_hand" in pd_info["info"].to_list():
        num_of_frames_missing_right_hand = pd_info.value_counts("info")[
            "missing_right_hand"]

    return [video_name,
            num_of_frames_total,
            num_of_frames_missing_pose,
            num_of_frames_missing_left_hand,
            num_of_frames_missing_right_hand]


In [10]:
analyze_results = []

for idx, video in enumerate(mp4_files):
    print(f"{idx}/{len(mp4_files)} - Analyzing {video}")
    statistics = get_analyze_statistics(video)
    statistics.append(GLOBAL_CONFIG)
    
    analyze_results.append(statistics)

print(len(analyze_results))

3192/3395 - Analyzing dataset/corpus/KIZ_0.mp4


I0000 00:00:1711917279.335104       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


3193/3395 - Analyzing dataset/corpus/ATILMAK_0.mp4


I0000 00:00:1711917283.091380       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


3194/3395 - Analyzing dataset/corpus/DINLEMEK_0.mp4


...

3393/3395 - Analyzing dataset/corpus/SEBEP_0.mp4


I0000 00:00:1711918229.931707       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


3394/3395 - Analyzing dataset/corpus/MECBUR_0.mp4


I0000 00:00:1711918233.809331       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


3395


In [13]:
pd_analyze_results = pd.DataFrame(analyze_results,
                                  columns=["video_name", 
                                           "num_of_total_frames", 
                                           "num_of_missing_frames_pose",
                                           "num_of_missing_frames_left_hand",
                                           "num_of_missing_frames_right_hand",
                                           "configuration"
                                           ])
pd_analyze_results

,video_name,num_of_total_frames,num_of_missing_frames_pose,num_of_missing_frames_left_hand,num_of_missing_frames_right_hand,configuration
0,dataset/corpus/GEREK _1.mp4,51,0,51,10,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
1,dataset/corpus/I╠çLGINC╠ğ_0.mp4,56,0,56,30,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
2,dataset/corpus/MEMUR_1.mp4,60,0,60,32,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
3,dataset/corpus/BAS╠ğPARMAK_0.mp4,41,0,41,19,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
4,dataset/corpus/YILDIZ_0.mp4,66,0,21,22,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
...,...,...,...,...,...,...
3390,dataset/corpus/GIZLI_1.mp4,70,0,48,49,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
3391,dataset/corpus/I╠çYI_0.mp4,34,0,34,18,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
3392,dataset/corpus/KULAK_0.mp4,63,0,63,27,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
3393,dataset/corpus/SEBEP_0.mp4,45,0,21,17,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."


In [15]:
formatted_datetime = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Seperator intentionally given as semicolon ";"
# as configuration has comma ","
pd_analyze_results.to_csv(f"analyze/{formatted_datetime}.csv" ,sep=';')

# Optional, write configuration as separate file
# write_global_config_to_file(f"analyze/{formatted_datetime}_config.txt")